# Python 常见的一些并行计算库

前面简单了解了下python并行的基本概念，这里就记录下平常见到的较常用的一些库。主要参考了：

- [Numba](https://github.com/numba/numba)
- [Introduction to Numba](https://nyu-cds.github.io/python-numba/)
- [Speed Up your Algorithms Part 2 — Numba](https://towardsdatascience.com/speed-up-your-algorithms-part-2-numba-293e554c5cc1)
- [Dask Tutorial](https://github.com/dask/dask-tutorial)
- [Speeding up your Algorithms Part 4— Dask](https://towardsdatascience.com/speeding-up-your-algorithms-part-4-dask-7c6ed79994ef)
- [Robust correlations with xarray and dask](http://martin-jung.github.io/post/2018-xarrayregression/)

## Numba

Numba是Python的即时编译器，在使用NumPy数组和函数以及循环的代码上效果最佳。使用Numba的最常见方法是通过其装饰器集合，这些装饰器可应用于函数以指示Numba对其进行编译。调用Numba装饰函数时，它会被“即时”编译为机器代码以执行，并且全部或部分代码随后可以本机机器速度运行！

可以直接通过conda或者pip安装numba。Munba常作为一个核心工具，还有一些其他的包可以提供额外其他的功能：scipy，colorama，pyyaml，icc_rt，用到再补充。

Numba是否作用取决于代码的情况，如果代码是numerically orientated 的（做大量数学运算），使用NumPy很多和/或有很多循环，那么Numba通常是一个不错的选择。下面了解下Numba的JIT装饰器 @jit 最基本的知识，以尝试加快某些功能的运行速度，并验真有效的方法和无效的方法。

Numba无法理解Pandas，因此当函数中出现pandas，Numba只会通过解释器运行此代码，会增加Numba内部开销。

In [1]:
from numba import jit
import numpy as np

x = np.arange(100).reshape(10, 10)

@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
def go_fast(a): # Function is compiled to machine code when called the first time
    trace = 0.0
    for i in range(a.shape[0]):   # Numba likes loops
        trace += np.tanh(a[i, i]) # Numba likes NumPy functions
    return a + trace              # Numba likes NumPy broadcasting

print(go_fast(x))

[[  9.  10.  11.  12.  13.  14.  15.  16.  17.  18.]
 [ 19.  20.  21.  22.  23.  24.  25.  26.  27.  28.]
 [ 29.  30.  31.  32.  33.  34.  35.  36.  37.  38.]
 [ 39.  40.  41.  42.  43.  44.  45.  46.  47.  48.]
 [ 49.  50.  51.  52.  53.  54.  55.  56.  57.  58.]
 [ 59.  60.  61.  62.  63.  64.  65.  66.  67.  68.]
 [ 69.  70.  71.  72.  73.  74.  75.  76.  77.  78.]
 [ 79.  80.  81.  82.  83.  84.  85.  86.  87.  88.]
 [ 89.  90.  91.  92.  93.  94.  95.  96.  97.  98.]
 [ 99. 100. 101. 102. 103. 104. 105. 106. 107. 108.]]


什么是nopython模式？

Numba@jit装饰器从根本上以两种编译模式（nopython模式和object模式）运行。在上面go_fast的示例中，nopython=True是在@jit装饰器中设置的，它指示Numba以nopython模式运行。nopython编译模式的行为实质上是编译装饰后的函数，以便其完全运行而无需Python解释器的参与。这是使用Numba jit装饰器的推荐和最佳实践方法，因为它可以带来最佳性能。

如果在nopython模式下编译失败，则Numba可以使用object mode进行编译 ，如果未设置nopython=True，则这是@jit 装饰器的 后备模式（如下例use_pandas所示）。在这种模式下，Numba将识别可以编译的循环并将其编译为在机器代码中运行的函数，并将在解释器中运行其余代码。为了获得最佳性能，请避免使用此模式！

In [2]:
from numba import jit
import pandas as pd

x = {'a': [1, 2, 3], 'b': [20, 30, 40]}

@jit
def use_pandas(a): # Function will not benefit from Numba jit
    df = pd.DataFrame.from_dict(a) # Numba doesn't know about pd.DataFrame
    df += 1                        # Numba doesn't understand what this is
    return df.cov()                # or this!

print(use_pandas(x))

<ipython-input-2-1a34fc432c2d>:6: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "use_pandas" failed type inference due to: non-precise type pyobject
During: typing of argument at <ipython-input-2-1a34fc432c2d> (8)

File "<ipython-input-2-1a34fc432c2d>", line 8:
def use_pandas(a): # Function will not benefit from Numba jit
    df = pd.DataFrame.from_dict(a) # Numba doesn't know about pd.DataFrame
    ^

  @jit
C:\Users\hust2\.conda\envs\hydrus\lib\site-packages\numba\core\object_mode_passes.py:178: NumbaWarning: Function "use_pandas" was compiled in object mode without forceobj=True.

File "<ipython-input-2-1a34fc432c2d>", line 7:
@jit
def use_pandas(a): # Function will not benefit from Numba jit
^

  state.func_ir.loc))
C:\Users\hust2\.conda\envs\hydrus\lib\site-packages\numba\core\object_mode_passes.py:188: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, 

      a      b
a   1.0   10.0
b  10.0  100.0


如何衡量Numba的性能？

首先，回想一下，Numba必须在执行函数的机器代码版本之前针对给定的参数类型编译函数。但是，一旦编译完成，Numba会为呈现的特定类型的参数缓存函数的机器代码版本。如果使用相同类型再次调用，则它可以重用缓存的版本，而不必再次编译。

衡量性能时，一个真正常见的错误是不考虑上述行为，并使用一个简单的计时器对代码进行一次计时，该计时器包括在执行时间中编译函数所花费的时间。

衡量Numba JIT对您的代码的影响的一种好方法是使用timeit模块函数对执行时间进行计时，这些函数测量执行的多次迭代，因此可以使其适应第一次执行时的编译时间。

附带说明一下，如果需要考虑编译时间，则Numba JIT支持 在磁盘上缓存已编译函数，并且还具有“提前”编译模式。

In [3]:
from numba import jit
import numpy as np
import time

x = np.arange(100).reshape(10, 10)

@jit(nopython=True)
def go_fast(a): # Function is compiled and runs in machine code
    trace = 0.0
    for i in range(a.shape[0]):
        trace += np.tanh(a[i, i])
    return a + trace

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
go_fast(x)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
go_fast(x)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

Elapsed (with compilation) = 0.6189589500427246
Elapsed (after compilation) = 0.0009734630584716797


有多快？

假设Numba可以在nopython模式下运行，或者至少编译一些循环，它将把编译目标定位到您的特定CPU。加速因应用而异，但可以是一到两个数量级。Numba的 性能指南涵盖了获得额外性能的常见选项。

Numba如何工作？

Numba读取装饰函数的Python字节码，并将其与有关函数输入参数类型的信息结合起来。它会分析和优化代码，最后使用LLVM编译器库生成针对CPU能力定制的函数的机器代码版本。然后，每次调用函数时都会使用此编译版本。

虽然Numba装饰器不多，不过除了@jit，还是有别的，比如@njit,@vectorize等，用到再说。

GPU目标：

Numba可以针对Nvidia CUDA和AMD ROC GPU。可以使用纯Python编写内核，并让Numba处理计算和数据移动。

要使用CUDA的话，需要安装[CUDA SDK](https://developer.nvidia.com/cuda-downloads)，安装时候要参考[NVIDIA官网文档](https://docs.nvidia.com/cuda/cuda-installation-guide-microsoft-windows/index.html)，[并设置相应的环境](https://numba.pydata.org/numba-doc/dev/cuda/overview.html#cudatoolkit-lookup)，如果是Linux下，Numba会自动给检测全局安装的CUDA库。

如果想要更多地了解GPU编程相关的内容，还需要进一步地去了解CUDA的基本编程，这有需要对C有所认识，略微需要更多基础，所以视情况，以后再补充了。

先粘贴一点关于python CUDA编程的教程：

- [CUDA Tutorial](https://www.tutorialspoint.com/cuda/index.htm)
- [CUDA Tutorial](https://cuda-tutorial.readthedocs.io/en/latest/)
- [CUDA Tutorials](http://supercomputingblog.com/cuda-tutorials/)
- [GPU Accelerated Computing with Python](https://developer.nvidia.com/how-to-cuda-python)
- [Python, Performance, and GPUs](https://towardsdatascience.com/python-performance-and-gpus-1be860ffd58d)
- [Getting Started with GPU Computing in Anaconda](https://www.anaconda.com/blog/getting-started-with-gpu-computing-in-anaconda)

## Dask

示例代码都放在 7-parallel-programming/dask-tutorial-master 下了。

Dask在大于内存的数据集上提供多核执行。

可以从高和低的角度理解dask：

- 高层集合： Dask提供了高级Array，Bag和DataFrame集合，它们模仿NumPy，lists和Pandas，但可以在不适合主内存的数据集上并行操作。Dask的高级集合是大型数据集的NumPy和Pandas的替代品。
- 底层调度： Dask提供了 dynamic task schedulers 动态任务调度程序，这些schedulers可以并行执行任务图（这块和pytorch中的计算图有些类似，先构建好要算的结构，最后再一气算）。这些执行引擎为上述高级集合提供支持，但也可以为用户定义的自定义工作负载提供支持。这些调度程序的等待时间很短（大约1毫秒），并且努力在较小的内存占用空间中运行计算。Dask的schedulers是在复杂情况中直接使用threading或 multiprocessing库或其他任务计划系统（例如Luigi或IPython paralle）的替代方法。

不同的用户在不同的级别上进行操作，但了解两者都有帮助。dask-tutorial教程将交错在高层次使用dask.array和 dask.dataframe（偶数部分）和低层次使用dask graphs and schedulers之间。

为了运行 dask-tutorial 的代码，需要安装：

```Shell
conda install -c conda-forge numpy pandas h5py pillow matplotlib scipy toolz pytables snakeviz scikit-image dask distributed
```

在README里已经装过了，这里就不重复了。

如果只在一台电脑上使用dask，那么没必要进行任何配置。在单个电脑上，有Single machine scheduler，很容易用。当然也可以在集群上使用。更多内容参考7-parallel-programming/parallel-tutorial-master 和 dask-tutorial-master 下的内容。 